In [1]:
## Reading a PDf File
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('resume.pdf')
docs=loader.load()
docs

[Document(metadata={'producer': 'pdf-lib (https://github.com/Hopding/pdf-lib)', 'creator': 'pdf-lib (https://github.com/Hopding/pdf-lib)', 'creationdate': '2025-06-28T08:01:28+00:00', 'moddate': '2025-06-28T08:01:28+00:00', 'source': 'resume.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content="Abhiram Kumar Soni\n\ue3b8+919798330406\n\ue0acabhiramsoni1@gmail.com\nProfiles\nAbhiram Soni\nLinkedIn\nAbhiram Kumar\nGitHub\nEducation\nWoolf University\nMaster of Science in CS: AI & ML\n(Jan 2025- Present)\nStudent Card\nE&ICT IIT Guwahati - AlmaBetter\nAdvanced Certification in Full Stack Data\nScience & AI\n(Nov 2023 - Sept 2024)\nCertificate\nVinoba Bhave University\nBachelor of Science in Mathematics\n(2021-2024)\nRelevant coursework: Statistics,\nProbability, Linear Algebra\nProvisional Certificate\nSkills\nExpertise in Languages & Tools (x/5)\nPython - 4.0 || SQL - 4.5|| Tableau - 4.1 ||\nExcel - 4.2 || Power BI - 4.3 || GitHub - 4.0\nTools & Platforms\nJupyter Notebook

In [19]:
def clean_text(text):
    text = str(text).replace("\n", " ")
    text = " ".join(text.split())
    return text

# If docs is already a list of strings
docs = [clean_text(doc) for doc in docs]


In [72]:
print(docs)

["Abhiram Kumar Soni \ue3b8+919798330406 \ue0acabhiramsoni1@gmail.com Profiles Abhiram Soni LinkedIn Abhiram Kumar GitHub Education Woolf University Master of Science in CS: AI & ML (Jan 2025- Present) Student Card E&ICT IIT Guwahati - AlmaBetter Advanced Certification in Full Stack Data Science & AI (Nov 2023 - Sept 2024) Certificate Vinoba Bhave University Bachelor of Science in Mathematics (2021-2024) Relevant coursework: Statistics, Probability, Linear Algebra Provisional Certificate Skills Expertise in Languages & Tools (x/5) Python - 4.0 || SQL - 4.5|| Tableau - 4.1 || Excel - 4.2 || Power BI - 4.3 || GitHub - 4.0 Tools & Platforms Jupyter Notebook, Google Colab,VS-Code, PostgreSQL Libraries & Frameworks Pandas, NumPy, Scikit-learn, Matplotlib, Seaborn, LangChain, Hugging Face, Groq, Ollama, OpenAI Machine Learning Supervised & Unsupervised Learning, Decision Tree, Bagging and Boosting, Cross validation, NLP, Recommendation system, Time series Analysis Certifications Full Stack D

In [26]:
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Assuming `docs` is a list of strings

# Convert strings to Document objects
documents = [Document(page_content=doc) for doc in docs]

# Now split them
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
final_documents = text_splitter.split_documents(documents)

# View the result
final_documents


[Document(metadata={}, page_content='Abhiram Kumar Soni \ue3b8+919798330406 \ue0acabhiramsoni1@gmail.com Profiles Abhiram Soni LinkedIn Abhiram Kumar GitHub Education Woolf University Master of Science in CS: AI & ML (Jan 2025- Present) Student Card E&ICT IIT Guwahati - AlmaBetter Advanced Certification in Full Stack Data Science & AI (Nov 2023 - Sept 2024) Certificate Vinoba Bhave University Bachelor of Science in Mathematics (2021-2024) Relevant coursework: Statistics, Probability, Linear Algebra Provisional Certificate Skills Expertise in'),
 Document(metadata={}, page_content='Provisional Certificate Skills Expertise in Languages & Tools (x/5) Python - 4.0 || SQL - 4.5|| Tableau - 4.1 || Excel - 4.2 || Power BI - 4.3 || GitHub - 4.0 Tools & Platforms Jupyter Notebook, Google Colab,VS-Code, PostgreSQL Libraries & Frameworks Pandas, NumPy, Scikit-learn, Matplotlib, Seaborn, LangChain, Hugging Face, Groq, Ollama, OpenAI Machine Learning Supervised & Unsupervised Learning, Decision Tre

In [77]:
# from langchain_text_splitters import RecursiveCharacterTextSplitter

# text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
# final_documents=text_splitter.split_documents(docs)
# final_documents

In [27]:
import os
from dotenv import load_dotenv
load_dotenv()

HF_TOKEN = os.getenv("HF_TOKEN")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
os.environ["HF_TOKEN"] = HF_TOKEN
os.environ["GROQ_API_KEY"] = GROQ_API_KEY


In [28]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

c:\personal project\Job Recommendation System\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [29]:
from langchain_community.vectorstores import FAISS

vectorstore=FAISS.from_documents(final_documents,embeddings)
vectorstore

In [43]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
You are a job recommendation assistant. 
Based on the candidate's resume projects, experience provided in <context>, suggest ONE best-fit job title.
Include reasoning for why the job matches their profile.

<context>
{context}
</context>
""")


In [44]:
from langchain_groq import ChatGroq

In [45]:

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [46]:
retriever=vectorstore.as_retriever()

from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="\nYou are a job recommendation assistant. \nBased on the candidate's resume projects, experience provided in <context>, suggest ONE best-fit job title.\nInclude reasoning for why the job matches their profile.\n\n<context>\n{context}\n</context>\n"), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000019732A6A5D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000019732A6A120>, model_name='llama-3.1-8b-instant', temperature=1e-08, model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser(), kw

In [47]:
from langchain.chains import create_retrieval_chain

retrieval_chain=create_retrieval_chain(retriever,document_chain)
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001970F95C1A0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="\nYou are a job recommendation assistant. \nBased on the candidate's resume projects, experience provided in <context>, suggest ONE best-fit job

In [48]:
output=retrieval_chain.invoke({"input":"suggest the best job for the candidate based on the following information: {context}"})

In [40]:
from pprint import pprint
pprint(output)


{'answer': "Based on the candidate's resume and experience, I recommend the "
           'job title: **Data Scientist**.\n'
           '\n'
           'Reasoning:\n'
           '\n'
           '1. **Technical Skills**: The candidate has a strong foundation in '
           'programming languages (Python, SQL), data analysis tools (Tableau, '
           'Power BI, Excel), and machine learning libraries (Pandas, NumPy, '
           'Scikit-learn, Matplotlib, Seaborn). They also have experience with '
           'NLP and recommendation systems.\n'
           "2. **Education and Certifications**: The candidate has a Master's "
           'degree in CS: AI & ML and has completed advanced certifications in '
           'Full Stack Data Science & AI and Machine Learning Training.\n'
           '3. **Practical Experience**: The candidate has hands-on experience '
           'in data wrangling, exploratory data analysis, data testing, '
           'dashboarding, and capstone projects focused on 

In [ ]:
candidate_skills=llm.invoke(f"based on the this content {docs} extract only skills of the candidate and give me clean output without any number or bullet points or any other symbols except for comma between skills")
skills=clean_text(candidate_skills)
print(skills)

content='Python, SQL, Tableau, Excel, Power BI, GitHub, Jupyter Notebook, Google Colab, VS-Code, PostgreSQL, Pandas, NumPy, Scikit-learn, Matplotlib, Seaborn, LangChain, Hugging Face, Groq, Ollama, OpenAI, Supervised & Unsupervised Learning, Decision Tree, Bagging and Boosting, Cross validation, NLP, Recommendation system, Time series Analysis, Statistics, Probability, Linear Algebra, Logistic Regression, Random Forest, XGBoost, Confusion Matrix, ROC-AUC, StandardScaler, Cosine Similarity, Collaborative Filtering' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 130, 'prompt_tokens': 1222, 'total_tokens': 1352, 'completion_time': 0.132171016, 'prompt_time': 0.083619729, 'queue_time': 0.074842412, 'total_time': 0.215790745}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_c8fb515de2', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--96c394ab-d6c1-4db4-b139-31e0e412087c-0' usage_metadata={'input_tokens': 1222, 'o

In [ ]:
# import re
# import requests

# # 1️⃣ Extract the job title using regex
# match = re.search(r'\*\*(.*?)\*\*', output['answer'])
# job_title = match.group(1) if match else "Data Scientist"

# print(f"Extracted Job Title: {job_title}")

# # 2️⃣ Create search query (target LinkedIn jobs in Bangalore)
# search_query = f"{job_title} India, site:linkedin.com/jobs OR site:indeed.com"

# # 3️⃣ Serper API config
# API_KEY = "c955f6ef055b28f704fcb79f219959ba2f2d5d40"  # Replace with your key
# url = "https://google.serper.dev/search"
# headers = {
#     "X-API-KEY": API_KEY,
#     "Content-Type": "application/json"
# }
# payload = {
#     "q": search_query
# }

# # 4️⃣ Send request to Serper
# response = requests.post(url, headers=headers, json=payload)

# # 5️⃣ Parse response & print top job links
# if response.status_code == 200:
#     results = response.json()
#     if "organic" in results:
#         print("\nTop Matching Job Links:\n")
#         for item in results["organic"][:5]:  # Top 3 results
#             print(f"Title: {item.get('title')}")
#             print(f"Link: {item.get('link')}\n")
#     else:
#         print("No results found.")
# else:
#     print(f"Error: {response.status_code} - {response.text}")


In [ ]:
import re
import requests

# 1️⃣ Extract the job title using regex
match = re.search(r'\*\*(.*?)\*\*', skills)
job_title = match.group(1) if match else "Data Scientist"

print(f"Extracted Job Title: {job_title}")

# 2️⃣ Create search query (target LinkedIn jobs in Bangalore)
search_query = f"{job_title} India, site:linkedin.com/jobs OR site:indeed.com"

# 3️⃣ Serper API config
API_KEY = "c955f6ef055b28f704fcb79f219959ba2f2d5d40"  # Replace with your key
url = "https://google.serper.dev/search"
headers = {
    "X-API-KEY": API_KEY,
    "Content-Type": "application/json"
}
payload = {
    "q": search_query
}

# 4️⃣ Send request to Serper
response = requests.post(url, headers=headers, json=payload)

# 5️⃣ Parse response & print top job links
if response.status_code == 200:
    results = response.json()
    if "organic" in results:
        print("\nTop Matching Job Links:\n")
        for item in results["organic"][:5]:  # Top 3 results
            print(f"Title: {item.get('title')}")
            print(f"Link: {item.get('link')}\n")
    else:
        print("No results found.")
else:
    print(f"Error: {response.status_code} - {response.text}")


Extracted Job Title: Data Scientist

Top Matching Job Links:

Title: 76000+ Data Scientist jobs in India (4935 new) - Bengaluru - LinkedIn
Link: https://in.linkedin.com/jobs/data-scientist-jobs

Title: 4,000 Data Scientist Job Vacancies | Indeed
Link: https://in.indeed.com/q-data-scientist-jobs.html

Title: 28,000+ Data Science jobs in India - LinkedIn
Link: https://in.linkedin.com/jobs/data-science-jobs

Title: Data Scientist in India Jobs (with Salaries) | Indeed Canada
Link: https://ca.indeed.com/q-data-scientist-in-india-jobs.html

Title: 13,000+ Data Scientist Artificial Intelligence jobs in India (914 new)
Link: https://in.linkedin.com/jobs/data-scientist-artificial-intelligence-jobs

